<center><b>EE 538 Spring 2021<br>
Low-Noise Analog Circuit Design<br>
University of Washington Electrical & Computer Engineering</b></center>


<b>Instructor: Jason Silver<br>
Assignment #3 (10 points)<br>
Due Monday, April 26 (Submit on Canvas as a Jupyter Notebook)</b> 

*Please show your work*

<b>Problem 1: Low-noise common-emitter amplifier design

<center><img src="img/CE_EF_biasing.png" width=400 /></center>
<br>
<center><b>Figure 1a. Common-emitter amplifier with bias </center>

<center><img src="img/CE_EF_midband.png" width=250 /></center>
<br>
<center><b>Figure 1b. Equivalent "mid-band" circuit </center>

The common-emitter amplifier is often biased using the configuration shown in Figure 1a, where $R_{B1,2}$ are used to set the DC base voltage and $R_E$ is needed to stabilize the DC bias condition against variations in temperature and transistor parameters. $Q_2$ is added to form an emitter-follower for low output impedance. The DC emitter current of the common-emitter stage ($I_{E1}$ is determined by

\begin{equation}
I_E = \dfrac{V_B - V_{BE}}{R_E + \frac{R_B}{\beta_0+1}}
\end{equation}


where $R_B$ is the parallel combination of $R_{B1}$ and $R_{B2}$ and component values satisfy the conditions $V_B >> V_{BE}$  and $R_E >> R_B/(\beta_0 +1)$.

$C_{AC}$ and $C_{AC2}$ are used to realize low impedances at signal frequencies at the base and emitter nodes, resulting in the equivalent "mid-band" circuit shown in Figure 1b. 

<u>*Analysis*</u>

__a)__ Assuming $R_s = 0$ and ignoring transistor capacitances $C_\pi$ and $C_\mu$, use the small-signal equivalent circuit of the amplifier in Figure 1a to derive its transfer function in terms of $R$'s, $C$'s, and transistor parameters $g_m$, $r_o$, and $\beta_0$.

__b)__ Determine expressions for the input-referred noise sources $e_n$ and $i_n$ of the amplifier in Figure 1b assuming only thermal and shot noise generators (no $1/f$ noise or $f_T$-dependent shot noise).

<u>*Design*</u>

__c)__ Assuming $R_s$ = $100\Omega$, $r_b = 2\Omega$ and $\beta_0 = 250$, what is the minimum theoretical noise figure of the amplifier? What value of $I_{C1}$ does this correspond to? Assume a mid-band gain of the common-emitter stage of $40\text{dB}$ and an output impedance of the emitter-follower of $50\Omega$. 

__d)__ Design the amplifier (i.e. determine resistor values $R_{B1,2}$, $R_E$, $R_C$, and $R_{E2}$) for a mid-band gain of $40\text{dB}$ and the noise figure determined in part c. Use a supply voltage $V_{CC} = 9V$. Determine values for $C_{AC}$ and $C_{AC2}$ that ensure a $3\text{dB}$ highpass corner lower than $100\text{Hz}$.

Verify your design in Ltspice using the SPICE model of the 2SC3324 npn transistor from Toshiba. Include an image of your schematic (with DC node voltages and branch currents annotated) and plots of the frequency response, output noise, and noise figure as a function of frequency. Indicate the noise figure at $10\text{kHz}$.

In [1]:
# Imports
import os
import sys
import cmath
import math
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import ltspice
import sympy as sp
from sympy.utilities.lambdify import lambdify
from scipy import signal
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib.ticker import LogLocator

In [2]:
def read_ltspice(file_name,ftype='trans',units='db'):
    cols = []
    arrs = []
    with open(file_name, 'r',encoding='utf-8') as data:
        for i,line in enumerate(data):
            if i==0:
                cols = line.split()
                arrs = [[] for _ in cols]
                continue
            parts = line.split()
            for j,part in enumerate(parts):
                arrs[j].append(part)
    df = pd.DataFrame(arrs,dtype='float64')
    df = df.T
    df.columns = cols
    if ftype=='trans':
        return df
    elif ftype=='ac':
        if units=='db':
            for col in cols:
                if df[col].str.contains(',').all():
                    df[f'Mag_{col}'] = df[col].apply(lambda x: x.split(',')[0])
                    df[f'Mag_{col}'] = df[f'Mag_{col}'].apply(lambda x: x[1:-2])
                    df[f'Mag_{col}'] = df[f'Mag_{col}'].astype('float64')
                    df[f'Phase_{col}'] = df[col].apply(lambda x: x.split(',')[1])
                    df[f'Phase_{col}'] = df[f'Phase_{col}'].apply(lambda x: x[0:-2])
                    df[f'Phase_{col}'] = df[f'Phase_{col}'].astype('float64')
        if units=='cartesian':
            for col in cols:
                if df[col].str.contains(',').all():
                    df[f'Re_{col}'] = df[col].apply(lambda x: x.split(',')[0])
                    df[f'Re_{col}'] = df[f'Re_{col}'].astype('float64')
                    df[f'Im_{col}'] = df[col].apply(lambda x: x.split(',')[1])
                    df[f'Im_{col}'] = df[f'Im_{col}'].astype('float64')
        df['Freq.'] = df['Freq.'].astype('float64')
        return df
    else:
        print('invalid ftype')

In [3]:
def rnd(num,places,unit):
    if unit.lower()=='mhz':
        return round(num/(1e6),places)

__a)__ Assuming $R_s = 0$ and ignoring transistor capacitances $C_\pi$ and $C_\mu$, use the small-signal equivalent circuit of the amplifier in Figure 1a to derive its transfer function in terms of $R$'s, $C$'s, and transistor parameters $g_m$, $r_o$, and $\beta_0$.

$\begin{eqnarray}
&& Z_s &=& R_s + \frac{1}{sC_{AC}} \\[0.5em]
&& v_{be} &=& \frac{(R_{in} \parallel R_B)}{(R_{in} \parallel R_B) + Z_s}v_s \\[0.5em]
&& Z_E &=& R_E \parallel \frac{1}{sC_{AC}} \\[0.5em]
&& R_{in} &=& r_{\pi}(1 + g_m Z_E) \\[0.5em]
\end{eqnarray}$